**Imports**

numpy for arrays

os for file searching and path manipulation

rasterio for reading and writing .tif files.

In [2]:
import numpy as np
import os
import rasterio as rio

**Box and Point Class**

A class to better store bounds as a Box object, with a method to check whether a point is withinside a box.

In [10]:
class OptionalArgumenterror(Exception):
    pass

class Box:
    def __init__(self, bounds=None, left:int=0, bottom:int=0, right:int=0, top:int=0):
        if bounds is not None: self.left,self.bottom,self.right,self.top = bounds
        else: self.left,self.bottom,self.right,self.top = left,bottom,right,top
        self.width, self.height = self.right -self.left, self.top -self.bottom
    def __str__(self):        
        return "left:{} bottom:{} right:{} top:{} width:{} height:{}".format(
            self.left, self.bottom, 
            self.right, self.top, 
            self.width, self.height)
    def in_bounds(self, x, y, op='exc') -> bool:
        '''
        in_bounds gets a x- and y-coordinate from a point, and an optional string (either 'exc' or 'inc')
        and returns whether that points is inside this box (either excluding or including the border area)
        '''
        if op == 'exc':
            return self.left < x < self.right and self.bottom < y < self.top
        elif op == 'inc':
            return self.left <= x <= self.right and self.bottom <= y <= self.top
        else:
            raise OptionalArgumenterror(f"{op} is not allowed as an argument, use either 'inc' or 'exc'")

class Point:
    def __init__(self, x:int=0, y:int=0):
        self.x,self.y = x,y
    def __str__(self):        
        return "x:{} y:{}".format(
            self.x, self.y)
    
test_box = Box(left=0,bottom=0,right=10,top=10)
test_point = Point(5,5)
another_point = Point(5,10)

print(test_box.in_bounds(test_point.x,test_point.y))
print(test_box.in_bounds(another_point.x,another_point.y))
print(test_box.in_bounds(another_point.x,another_point.y, 'inc'))

True
False
True


**My own Cropper**

I didn't get it to work, so I used a slightly changed version of Maarten's subdivide, with some elements of my own try.

I actually lost the exact code I used to split the files in a bad git merge, so I thought it would be best to just show my broken code and the subdivide together, instead of trying to replicate something I made before.

In [3]:
def cropper(origin, new_name, x, y, new_width, new_height):
        origin_arr = np.array(origin.read(1))
        origin_bounds = origin.bounds
      
        left = int(x * width)
        right = int((x+1) * width)
        bottom = int((y+1) * height)
        top = int(y * height)
            
        new_left = origin_bounds.left + left
        new_right = origin_bounds.left + right
        new_bottom = origin_bounds.top - bottom
        new_top = origin_bounds.top - top
        
        print(f"[{x},{y}]: {new_left}{new_right}|{new_bottom}{new_top}")
        # crop out of the array
        new_arr = origin_arr[left:right,top:bottom]
        
        new_meta = origin.meta
        
        # transform needs to be edited so the crop has correct coords
        new_meta['transform'](
            # new left-most coord
            new_left,
            # new bottom-most coord
            new_bottom,
            # new right-most coord
            new_right,
            # new top-most coord
            new_top,
            # new width
            new_width,
            # new height
            new_height)

        new_meta["width"] = new_width
        new_meta["height"] = new_height

        new_path = os.path.join("/media/seppe/DiskSpaceInvader/3DHouseData",new_name)
        # open a new file in write mode as destination
        # **meta is the **kwargs passed to the tif
        with rio.open(new_path, "w", **new_meta) as destination:
            # write the pixel data with the **meta data
            destination.write(new_arr, indexes=1)
            print("new file: ",new_name)
            destination.close()
        return

**Maarten's Subdivide**

I will probably name my firstborn after Maarten. Without his feedback, I'd still be splitting .tif files.

In [ ]:
def subdivide(name:str, tif:rio.io.DatasetReader, sub:int=2) -> None:
    if sub is 0: return
    
    arr = np.array(tif.read(1))
    meta = tif.meta
    bounds = tif.bounds
    
    meta["width"]  /= 2
    meta["height"] /= 2
    
    sub_bound = {
        0:(0,1,-1,0),
        1:(1,1,0,0),
        2:(0,0,-1,-1),
        3:(1,0,0,-1)}
    
    sub_slice = {
        0:{"sx":slice(None,int(meta["height"])),
           "sy":slice(None,int(meta["width"]))},
        1:{"sx":slice(None,int(meta["height"])),
           "sy":slice(int(meta["width"]),None)},
        2:{"sx":slice(int(meta["height"]),None),
           "sy":slice(None,int(meta["width"]))},
        3:{"sx":slice(int(meta["height"]),None),
           "sy":slice(int(meta["width"]),None)}}
    
    for idx in range(4):        
        meta["transform"] = from_bounds(
            bounds.left + meta["width"] * sub_bound[idx][0],
            bounds.bottom + meta["height"] * sub_bound[idx][1],
            bounds.right + meta["width"] * sub_bound[idx][2],
            bounds.top + meta["height"] * sub_bound[idx][3],
            meta["width"], meta["height"]
        )
        with rio.open(name +f"_{idx}.tif", "w+", **meta) as subdiv:
            subdiv.write(arr[sub_slice[idx]["sx"],sub_slice[idx]["sy"]], indexes=1)
            subdivide(name +f"_{idx}", subdiv, sub-1)
            if sub > 1: os.remove(name +f"_{idx}.tif")
            else: sub_lookup[name[2:] +f"_{idx}"] = Box(subdiv.bounds)

sub_lookup = {} # !!! Should write to dataframe directly

**File Searcher**

loops through all files in a certain path, and applies a given function to every file of the desired type

In [5]:
# get the relative path of the current project that this notebook is in
project_path = os.path.abspath(os.path.join("readfiles.ipynb", os.pardir, os.pardir))
print("Found project in: ", project_path)
data_path = os.path.join(project_path, "Data")

external_data_path = "/media/seppe/DiskSpaceInvader/3DHouseData"


def search_files(fpath: str, file_type: str, func):
    '''
    Walk through the folder containing the data, and check if the files match the file type
    If so; open that file, and print how many there were found
    '''
    total_files = 0
    for root, dirs, files in os.walk(fpath):
        for name in files:
            if name.endswith(file_type):
                func(root + "/" + name)
                total_files += 1
    print(f"Found {total_files} {file_type} files.")
    return


all_tifs = []
search_files(external_data_path, ".tif", all_tifs.append)
print(len(all_tifs))

Found project in:  /home/seppe/Projects/BeCode/3D-House-Project
Found 88150 .tif files.
88150


In [ ]:
# a function to pass as an argument to the search_files function
def tifsplit(filepath):
    '''
    Call the subdivide fuction, with the new name, opened tif, and do 5 recursion divides on every file.
    This will split every file into 1024 smaller files.
    '''
    tif = rio.open(filepath)
    
    # change name from /media/ ... /GeoTIFF/DHMVIIDSMRAS1m_k16.tif
    #               to /media/ ... /GeoTIFF/DHMVIIDSMRAS1m_k16
    name = filepath[:-4]
    subdivide(name, tif, 5)

# call searchfiles, and do a tifsplit on every tif file in the data path    
search_files(external_data_path, ".tif", tifsplit)